<a href="https://colab.research.google.com/github/ngockhanh5110/nlp-vietnamese-text-summarization/blob/main/notebooks/testing_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall transformers
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!pip3 install --upgrade .
%cd '/content/'

In [ ]:
!wget 'https://github.com/CLC-HCMUS/ViMs-Dataset/raw/master/ViMs.zip'
!unzip 'ViMs.zip'

In [ ]:
!pip install datasets==1.0.2

In [ ]:
import glob
import pandas as pd
import concurrent.futures
import datasets

In [ ]:
# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

# Processing data

In [ ]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

In [ ]:
pathfiles = list()
for pathdir in glob.glob('/content/ViMs/original/*'):
  for pathfile in glob.glob(pathdir + '/original/*'):
    pathfiles.append(pathfile)

In [ ]:
def read_content(pathfile):
  """
  Input: Path of txt file
  Output: A dictionary has keys 'original' and 'summary'
  """
  with open(pathfile) as f:
    rows  = f.readlines()
    original = ''
    summary = ''
    start_copy_summary = False
    start_copy_content= False

    for row in rows:
      if row[:7] == 'Content':
        start_copy_summary = False
        start_copy_content = True
      elif start_copy_content:
        original += row + ' '
      elif start_copy_summary:
        summary += row + ' '
      elif row[:7] == 'Summary': 
        start_copy_summary = True
        summary += row[9:] + ' '

    original = rdrsegmenter.tokenize(original)
    original = ' '.join([' '.join(x) for x in original])

    summary = rdrsegmenter.tokenize(summary)
    summary = ' '.join([' '.join(x) for x in summary])

    if summary == '':
      summary = None
    if original == '':
      original = None
      
    return {'file' : pathfile,
            'original': original, 
            'summary': summary}

In [ ]:
read_content('/content/ViMs/original/Cluster_001/original/8.txt')

{'file': '/content/ViMs/original/Cluster_001/original/8.txt',
 'original': 'Bộ_trưởng Quốc_phòng Hy_Lạp Panos_Kammenos , máy_bay đã " quay 90 độ sang trái và 360 độ sang phải , hạ độ cao từ 37.000 feet xuống 15.000 feet và mất tín_hiệu ở độ cao 10.000 feet " ( khoảng 11 km ) . Theo lời quan_chức này , chiếc máy_bay MS804 bất_ngờ bay chậm lại ở độ cao khoảng hơn 11km . Ở toạ_độ này , máy_bay " bất_ngờ chuyển_hướng " , ban_đầu xoay 90 độ sang bên phải , sau đó lại xoay ngược về bên trước khi biến mất khỏi màn_hình radar và có_thể là bắt_đầu rơi . Theo nguồn tin sân_bay Hy_Lạp cho biết , chiếc máy_bay MS804 đã rơi ở ngoài khơi đảo Karpathos của nước này . Truyền_thông Hy_Lạp cũng đưa tin rằng , ngư_dân trên một tàu đánh_cá của Hy_Lạp đã nhìn thấy một vệt sáng trên bầu_trời ở Địa_Trung_Hải . Tuy_nhiên , thông_tin này hiện chưa được kiểm_chứng . Một đoạn video xuất_hiện trên mạng Twitter được cho là quay lại khoảnh_khắc máy_bay của hãng hàng_không EgyptAir bốc cháy như một quả cầu lửa trên 

In [ ]:
with concurrent.futures.ProcessPoolExecutor() as executor:
  data = executor.map(read_content, pathfiles)

In [ ]:
# Make blank dataframe
data_df = list()
for d in data:
  data_df.append(d)
data_df = pd.DataFrame(data_df)
data_df

,file,original,summary
0,/content/ViMs/original/Cluster_040/original/26...,"Ngày 20-5 , Tỉnh_đoàn Khánh_Hoà tổ_chức lễ tra...",None
1,/content/ViMs/original/Cluster_040/original/26...,"Ngày 20/5 , Tỉnh đoàn Khánh Hòa đã trao h...",Hai thiếu_niên người dân_tộc Raglai dũng_cảm c...
2,/content/ViMs/original/Cluster_040/original/25...,"Đó là 2 em Bo_Tý ( 14 tuổi , trú xã Ba Cụm Bắc...","Sáng nay 20/5 , tại Tỉnh_Đoàn Khánh_Hoà đã diễ..."
3,/content/ViMs/original/Cluster_040/original/26...,"Thông_tin trên Thanh_niên cho biết , sáng 20/5...","Trung_ương Đoàn đã trao huy_hiệu "" Tuổi_trẻ dũ..."
4,/content/ViMs/original/Cluster_040/original/25...,"Sáng nay 20.5 , Tỉnh_đoàn Khánh_Hoà tổ_chức lễ...",Hai thiếu_niên người dân_tộc Raglai dũng_cảm c...
...,...,...,...
1940,/content/ViMs/original/Cluster_036/original/23...,"Ngày 19.5 , ông Nguyễn_Như_Tiệp , Cục trưởng C...",Bộ Nông_nghiệp Phát_triển nông_thôn yêu_cầu tỉ...
1941,/content/ViMs/original/Cluster_036/original/23...,"Ngày 19/5 , Cục Quản_lý chất_lượng nông , lâm ...",Chi_cục quản_lý chất_lượng nông - lâm - thuỷ_s...
1942,/content/ViMs/original/Cluster_036/original/23...,"Công_văn nêu rõ , trong thời_gian qua đã xảy r...",Cục Quản_lý Chất_lượng Nông_Lâm và Thuỷ_sản ( ...
1943,/content/ViMs/original/Cluster_036/original/23...,Cục Quản_lý Chất_lượng Nông_lâm_sản và Thuỷ_sả...,None


In [ ]:
data_df.dropna(inplace = True)
data_df

NameError: ignored

# DataLoader

# Modelling

In [ ]:
import torch
from transformers import EncoderDecoderModel , BertTokenizer, BertGenerationEncoder, BertGenerationDecoder, AutoTokenizer

encoder = BertGenerationEncoder.from_pretrained("vinai/phobert-base", bos_token_id=101, eos_token_id=102)
# add cross attention layers and use BERT's cls token as BOS token and sep token as EOS token
decoder = BertGenerationDecoder.from_pretrained("vinai/phobert-base", add_cross_attention=True, is_decoder=True, bos_token_id=101, eos_token_id=102)
bert2bert = EncoderDecoderModel(encoder=encoder, decoder=decoder)

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing BertGenerationEncoder: ['roberta.embeddings.position_ids', 'roberta.embeddings.word_embeddings.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.value.weight', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encoder.layer.0.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.encoder.layer.0

In [ ]:
input_ids = tokenizer('Tôi là sinh_viên trường đại_học Công_nghệ Việt_Nam.', add_special_tokens=False, return_tensors="pt").input_ids
labels = tokenizer('sinh_viên đại_học Công_nghệ.', return_tensors="pt").input_ids

# train...
loss = bert2bert(input_ids=input_ids, decoder_input_ids=labels, labels=labels).loss
loss.backward()